# Pre-Trained BERT 12-Layer Model





In [ ]:
# Loss function - Cross Entropy loss
# Optimizer - AdamW

!pip install transformers
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm, trange
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, AdamW
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Fetching the CUDA device name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.get_device_name(0)

cuda


'Tesla K80'

In [ ]:
filePath = 'https://raw.githubusercontent.com/serenpa/Blog-Credibility-Corpus/main/blog_credibility_dataset.tsv'
df = pd.read_csv(filePath, sep='\t', encoding='latin1')
df['sentence_text'] = df.sentence_text.apply(lambda x:x.encode('latin1').decode('cp1252'))
samSize = len(df[df.Experience==1])
df_sampled = pd.concat([df[df.Experience==0].sample(samSize, random_state=50), df[df.Experience==1]])
df_sampled = df_sampled.sample(frac=1,random_state=50).reset_index(drop=True)
df_sampled.Experience.value_counts()
# Creating a 'list' of sentences using data frame by selecting only values
sentences = df_sampled.sentence_text.values

BERT takes in data in a specific format, and thats the reason we need to tokenize the input sentences.
Also, the sentence sequence is maintained using input ids, attention masks were created to allow BERT to learn the importance of each sentence by itself.

In [ ]:
# Formatting the sentences as required
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df_sampled.Experience.values
unique, counts = np.unique(labels, return_counts=True)
dict(zip(unique, counts))

{0: 2590, 1: 2590}

In [ ]:
# Tokenizing the sentences using BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(sentences[0])
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

[CLS] I told them about the 45 bugs. [SEP]
Tokenize the first sentence:
['[CLS]', 'i', 'told', 'them', 'about', 'the', '45', 'bugs', '.', '[SEP]']


Having a fixed length input arrays organizes the BERT learning in a structured manner and we have introduced maximum sentence length to fix the input array length. Padding is used to maintain this fixed length across the sentences. 

In [ ]:
# BERT tokenizer to convert tokens to their respectively index number as per BERT vocabulary
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
print(len(input_ids[0]))

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(len(input_ids[0]))

10
128


In [ ]:
# Attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
# Creating tensor variables 

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=50, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=50, test_size=0.2)

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
# creating tensor datasets

batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In this step, we are loading the BERT.

There are a few different pre-trained BERT models available. We have used "bert-base-uncased" version that has only lowercase letters ("uncased") and is the smaller version of the two ("base" vs "large").

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Now that we have our model loaded we need to grab the training hyperparameters from within the stored model.

Fine-tuning is also carried out in this step. Experts recommend the following hyperparameter ranges and the same has been used here:
- Batch size: 16
- Learning rate (Adam): 5e-5, 3e-5, 2e-5
- Number of epochs: 4, 5, 6



In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Within our training loop, we take care of the below:
- Tell the model to compute gradients by setting the model in train mode
- Unpack our data inputs and labels
- Load data onto the GPU for acceleration
- Clear out the gradients calculated in the previous pass. In pytorch the gradients accumulate by default (useful for things like RNNs) unless you explicitly clear them out
- Forward pass (feed input data through the network)
- Backward pass (backpropagation)
- Tell the network to update parameters with optimizer.step()
- Track variables for monitoring progress

And during the evaluation loop:
- Tell the model not to compute gradients by setting th emodel in evaluation mode
- Unpack our data inputs and labels
- Load data onto the GPU for acceleration
- Forward pass (feed input data through the network)
- Compute loss on our validation data and track variables for monitoring progress

In [ ]:
t = [] 
trueLabels = []
predLabels = []
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 5
loss=torch.nn.CrossEntropyLoss()
# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  # Training
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # batch = tuple(t for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids.long(), token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)[0]

    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))
  # Validation
  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    #batch = tuple(t for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids.long(), token_type_ids=None, attention_mask=b_input_mask)[0]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    trueLabels.extend(label_ids)
    predLabels.extend(np.argmax(logits, axis=1).flatten())

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.5561931884081668


Epoch:  20%|██        | 1/5 [03:13<12:55, 193.95s/it]

Validation Accuracy: 0.7932692307692307
Train loss: 0.40815859100993535


Epoch:  40%|████      | 2/5 [06:28<09:43, 194.53s/it]

...

Validation Accuracy: 0.7573717948717948
Train loss: 0.15199009731814667


Epoch:  80%|████████  | 4/5 [13:01<03:15, 195.67s/it]

Validation Accuracy: 0.7592948717948718
Train loss: 0.07228410302690126


Epoch: 100%|██████████| 5/5 [16:16<00:00, 195.27s/it]

Validation Accuracy: 0.7564102564102564


In [ ]:
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Validation Accuracy: 0.7564102564102564


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(trueLabels, predLabels))

              precision    recall  f1-score   support

           0       0.77      0.78      0.78      2655
           1       0.77      0.76      0.76      2525

    accuracy                           0.77      5180
   macro avg       0.77      0.77      0.77      5180
weighted avg       0.77      0.77      0.77      5180

